<a href="https://colab.research.google.com/github/etuckerman/surf_NLP/blob/main/surf_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Log into HuggingFace**

In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
!pip install pandas
!pip install datasets evaluate [sentencepiece]
! pip install -U accelerate
! pip install -U transformers
from transformers import Trainer

ERROR: Invalid requirement: '[sentencepiece]'


In [3]:
import pandas as pd
from datasets import Dataset


In [4]:
data_path = "/content/surf_messages.csv"  # Replace with your CSV path


In [5]:
df = pd.read_csv(data_path)


In [6]:
# Add your data cleaning steps here (if needed)
# For example, convert messages to lowercase
df["Message"] = df["Message"].str.lower()


In [7]:
def prepare_data(data):
  inputs = tokenizer(data["Message"], padding="max_length", truncation=True)
  labels = data["Label"]  # Assuming you have a "Label" column with intent labels
  return inputs, labels

dataset = Dataset.from_pandas(df.drop("Author", axis=1))  # Drop "Author" column and use "fn" for prepare_data



In [8]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "distilbert-base-uncased"  # Replace with your chosen model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(set(dataset["Message"])))  # Adjust num_labels for your intent classes


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",  # Output directory for checkpoints
    per_device_train_batch_size=8,  # Batch size
    num_train_epochs=3,  # Training epochs
)


In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],  # Assuming your dataset is split into train/validation sets
)


KeyError: "Column train not in the dataset. Current columns in the dataset: ['Message']"

In [ ]:
trainer.train()


In [ ]:
trainer.save_model("./saved_model")  # Save model and tokenizer to a directory
